```mermaid
flowchart LR
    A[dataset_builder] -->|原始样本| B[teacher_labeler]
    B -->|生成标签| C[train_lora]
    C -->|LoRA 模型| D[evaluate]
```

以上示意图展示了各模块的协作关系：
- `dataset_builder`：收集与清洗数据，输出原始训练样本；
- `teacher_labeler`：调用教师模型，为样本添加 `prediction`、`analysis` 等标签；
- `train_lora`：基于带标签数据执行 LoRA 微调；
- `evaluate`：加载微调模型，在验证集上评估效果。

### LoRA 原理概述
LoRA（Low-Rank Adaptation）将权重增量表示为低秩分解 $\Delta W = B A$，仅训练小矩阵 $A,B$，而预训练权重 $W_0$ 保持冻结不更新，从而在保持原模型能力的同时大幅减少可训练参数和显存开销。

在本示例中，可选参数 `rope_factor` 会按比例缩放旋转位置编码（RoPE）的基础频率，以此扩展模型可接受的上下文长度；例如 `rope_factor=2` 时，理论上可将上下文窗口扩大到原来的两倍。

### 金融场景下的动机示例
在金融任务中，模型需理解长篇财报、行情序列并生成投资建议。通过 LoRA 微调，我们只需在少量行业数据上训练小规模适配器，即可快速获得专用模型；配合 `rope_factor` 扩展后的 RoPE，模型还能一次性读入更长的时间区间或多份文档。这为后续的数据构建、教师标注、训练与评估提供了现实动机。

# 大模型 LoRA 微调流程演示

本示例 Notebook 展示金融股票领域大模型微调的完整流程，包括数据集构建、教师模型标注、LoRA 微调训练以及模型评估。示例使用 `test4` 模块中的代码，并利用模拟数据和模型以便在轻量环境下演示整体流程。

## 背景介绍
- **LoRA** 通过在预训练模型的部分权重上插入低秩矩阵，在冻结原模型参数的情况下完成高效微调。
- **RoPE 长上下文扩展**: 通过设置 `rope_factor` 扩展旋转位置编码，使模型支持更长的输入序列。
- **Token 截断**: 在构造数据集和训练时确保输入长度不超过最大上下文窗口。

## 运行前准备
- 安装依赖：`pip install -r requirements.txt`
- 建议执行顺序：数据集构建 → 教师标注 → LoRA 训练 → 评估。
- CPU 环境提示：示例可在无 GPU 条件下运行，但速度较慢。
- 常见错误排查：
  - `ModuleNotFoundError`：确认依赖已安装。
  - 路径不存在：检查 `data_path`、`output_dir` 等参数。
  - 显存不足：减小 `batch_size` 或切换到 CPU。


## 步骤1：构建模拟数据集
下面构造两只股票的30日K线数据，并使用 `dataset_builder.build_dataset` 生成训练/验证样本。为避免真实网络请求，我们 monkey-patch `_fetch_kline` 函数返回模拟数据。

In [ ]:
import os, sys, json
sys.path.append(os.getcwd())  # 使 test4 模块可导入

from test4 import dataset_builder

import pandas as pd
import numpy as np

dates = pd.date_range('2023-01-01', periods=30, freq='D').strftime('%Y-%m-%d')
prices1 = 100 * np.cumprod(1 + np.random.normal(0, 0.01, size=30))
prices2 = 50 * np.cumprod(1 + np.random.normal(0, 0.01, size=30))

df_stock1 = pd.DataFrame({
    'date': dates,
    'open': prices1,
    'close': prices1 + np.random.normal(0, 0.5, size=30),
    'high': prices1 + np.random.normal(0, 1, size=30),
    'low':  prices1 - np.random.normal(0, 1, size=30),
    'volume': np.random.uniform(1000, 10000, size=30)
})

df_stock2 = pd.DataFrame({
    'date': dates,
    'open': prices2,
    'close': prices2 + np.random.normal(0, 0.5, size=30),
    'high': prices2 + np.random.normal(0, 1, size=30),
    'low':  prices2 - np.random.normal(0, 1, size=30),
    'volume': np.random.uniform(1000, 10000, size=30)
})

def dummy_fetch_kline(code: str, days: int):
    if code == '000001':
        return df_stock1.tail(days).reset_index(drop=True)
    elif code == '000002':
        return df_stock2.tail(days).reset_index(drop=True)
    else:
        return None

dataset_builder._fetch_kline = dummy_fetch_kline

train_samples, val_samples = dataset_builder.build_dataset(['000001','000002'], days=30, window=30, val_ratio=0.2, seed=42)
print(f"训练样本数: {len(train_samples)}, 验证样本数: {len(val_samples)}")
if train_samples:
    example_prompt = dataset_builder.format_prompt(train_samples[0])
    print(example_prompt[:120] + '...')

In [ ]:
from collections import Counter
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained('sshleifer/tiny-gpt2')
prompt_tokens = tok(dataset_builder.format_prompt(train_samples[0]), add_special_tokens=False)['input_ids']
sample_copy = train_samples[0].copy()
dataset_builder._trim_sample_tokens(sample_copy, tok, max_tokens=50)
trimmed_tokens = tok(dataset_builder.format_prompt(sample_copy), add_special_tokens=False)['input_ids']
print('kline_summary 首条记录:', json.dumps(train_samples[0]['kline_summary'][:2], ensure_ascii=False))
cats = Counter('up' if s['change'] > 3 else 'down' if s['change'] < -3 else 'stable' for s in train_samples)
print('类别分布:', cats)
print('修剪前 token 数:', len(prompt_tokens), '修剪后 token 数:', len(trimmed_tokens))


上述输出展示了 `build_dataset` 的几个关键特性：
- `_fetch_kline` 被 monkey-patch 为返回模拟 K 线数据；实际使用时可替换为真实行情接口。
- 构造过程中计算了 `MA5`、`MA10`、`RSI14`、`MACD` 等技术指标，并通过滑窗抽取样本。
- 若 `balance_classes=True`，会上下平衡 `up`/`down`/`stable` 三类样本数量。
- 通过传入 `tokenizer` 与 `max_tokens` 参数，可触发 `_trim_sample_tokens` 对过长的 `kline_summary` 进行截断，上例展示了截断前后 token 数的差异。


## 步骤2：教师模型标注
使用 `teacher_labeler` 将 Prompt 列表转换为带有 `prediction`、`analysis` 和 `advice` 字段的答案。默认的 `call_teacher` 接口接受一个字符串 Prompt，返回包含模型输出的 JSON 字符串或 `{'content': ...}`。在 Notebook 中我们以 `dummy_call_teacher` 进行演示，可替换为真实的教师模型调用。


In [ ]:
from test4 import teacher_labeler

def dummy_call_teacher(prompt: str):
    if '000001' in prompt:
        pred = '上涨'
    else:
        pred = '下跌'
    return json.dumps({'prediction': pred, 'analysis': '模拟分析', 'advice': '模拟建议'}, ensure_ascii=False)

teacher_labeler.call_teacher = dummy_call_teacher

train_prompts = [dataset_builder.format_prompt(s) for s in train_samples]
val_prompts = [dataset_builder.format_prompt(s) for s in val_samples]

train_records = teacher_labeler.label_samples(train_prompts, output_file='labeled_data.jsonl')
val_records = teacher_labeler.label_samples(val_prompts, output_file='val_labeled_data.jsonl')

print(f"已标注训练样本数: {len(train_records)}, 验证样本数: {len(val_records)}")
print(json.dumps(train_records[0], ensure_ascii=False, indent=2))

In [ ]:
def noisy_call_teacher(prompt: str):
    if '坏' in prompt:
        return 'not a json'
    else:
        return '{"prediction": "上涨"}'
teacher_labeler.call_teacher = noisy_call_teacher
bad_records = teacher_labeler.label_samples(['坏 JSON', '缺字段'], output_file='bad_labeled_data.jsonl')
print(json.dumps(bad_records, ensure_ascii=False, indent=2))
teacher_labeler.call_teacher = dummy_call_teacher


上例显示 `label_samples` 对异常返回的清洗逻辑：非法 JSON 被保存到 `raw` 字段，而缺失字段会自动补为空字符串。标注后的数据会写入 `labeled_data.jsonl` 等路径，供训练与评估复用。


## 步骤3：LoRA 微调训练
下面调用 `train_lora.main` 进行一次简化训练。为了便于快速运行，使用体积很小的 `sshleifer/tiny-gpt2` 作为基础模型，并将训练步数限制为 1。


In [ ]:
from test4 import train_lora

import os, torch, random, numpy as np
torch.manual_seed(0); random.seed(0); np.random.seed(0)

if not hasattr(train_lora, 'orig_BitsAndBytesConfig'):
    train_lora.orig_BitsAndBytesConfig = train_lora.BitsAndBytesConfig

def safe_bnb_config(**kwargs):
    try:
        import torch
        if torch.cuda.is_available():
            return train_lora.orig_BitsAndBytesConfig(**kwargs)
    except ImportError:
        pass
    return None

train_lora.BitsAndBytesConfig = safe_bnb_config

cfg = train_lora.TrainConfig(
    base_model='sshleifer/tiny-gpt2',
    data_path='labeled_data.jsonl',
    eval_path='val_labeled_data.jsonl',
    output_dir='lora_demo',
    epochs=1,
    max_steps=1,
    batch_size=1,
    max_len=4096,
    rope_factor=2.0
)

train_lora.main(cfg)
print('LoRA 微调完成，输出目录：', os.listdir('lora_demo'))


## 步骤4：模型评估
使用 `test4.evaluate.evaluate_model` 统一计算 BLEU、ROUGE-L 及向量相似度指标，并与基座模型作对比。下方保留原始的 BLEU 计算示例以便理解评估流程。


In [ ]:
from test4 import evaluate
if not hasattr(evaluate, 'orig_BitsAndBytesConfig'):
    evaluate.orig_BitsAndBytesConfig = evaluate.BitsAndBytesConfig

def safe_bnb_config(**kwargs):
    try:
        import torch
        if torch.cuda.is_available():
            return evaluate.orig_BitsAndBytesConfig(**kwargs)
    except ImportError:
        pass
    return None

evaluate.BitsAndBytesConfig = safe_bnb_config

prompts = [rec['prompt'] for rec in val_records]
refs = [json.dumps(rec['label'], ensure_ascii=False, sort_keys=True) for rec in val_records]
base_scores = evaluate.evaluate_model('sshleifer/tiny-gpt2', prompts, refs)
tuned_scores = evaluate.evaluate_model('lora_demo', prompts, refs)
print('Base model:', base_scores)
print('LoRA model:', tuned_scores)

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
preds = [json.dumps(rec['label'], ensure_ascii=False, sort_keys=True) for rec in val_records]
smooth = SmoothingFunction().method4
bleu = [sentence_bleu([ref.split()], pred.split(), smoothing_function=smooth) for ref, pred in zip(refs, preds)]
print('手工 BLEU 示例:', sum(bleu) / len(bleu))


## 总结
通过以上步骤，我们演示了基于 `test4` 模块的完整 LoRA 微调流程：从构建股票K线数据集、教师模型标注，到对学生模型进行 LoRA 微调并进行指标评估。该流程能够在有限资源下让模型学习金融领域知识。